# Titanlib min working example

### Table of Contents  

- [How to make a titanlib point](#How-to-make-a-titanlib-point)
- [How to make a dataset](#How-to-make-a-dataset)
- [Buddy check](#Buddy-check)
- [Buddy event check](#Buddy-event-check)
- [Climatology range check](#Climatology-range-check)
- [First guess test](#First-guess-test)
- [Isolation check](#Isolation-check)
- [Metadata check](#Metadata-check)
- [Range check](#Range-check)
- [Spatial consistency test](#Spatial-consistency-test)
- [Spatial consistency test resistant](#Spatial-consistency-test-resistant)
- [Spatial consistency test dual](#Spatial-consistency-test-dual)




## How to make a titanlib point

In [1]:
import titanlib
import xarray as xr
import pandas as pd
import os
import numpy as np

lats = [57.2, 57.21, 50.5, 11.2]
lons =  [10.3, 10.31, 13.2, 29.5]
elevs = [50, 51, 912, 31]
observations = [20, 30, 18, 25]
points = titanlib.Points(lats, lons, elevs)
flags = titanlib.metadata_check(
    points, True, True, True, True
)


## How to make a dataset

This is a way that titanlib manages multiple checks preverving the flags.

In [2]:
dataset = titanlib.Dataset(points, observations)

tmin = [-20,-20,-20,-20]
tmax = [30,30,30,30]
radius = 1500000 # m
nb_neigh = 1


dataset.range_check(tmin, tmax)
dataset.isolation_check(nb_neigh, radius)
print(dataset.get_flags())


[0 0 0 1]


## Buddy check

In [3]:
#Buddy check 

radius = np.full(points.size(), 5000)
num_min = np.full(points.size(), 1)
threshold = 2
max_elev_diff = 200
elev_gradient = -0.0065
min_std = 1
num_iterations = 5

flags = titanlib.buddy_check(
    points,
    observations,
    radius,
    num_min,
    threshold,
    max_elev_diff,
    elev_gradient,
    min_std,
    num_iterations,
)

print(flags)

[1 1 0 0]


## Buddy event check  

In [4]:
#Buddy event check

precip_obs = [5, 0.1, 2, 0]
radius = np.full(points.size(), 5000)
num_min = np.full(points.size(), 1)
event_threshold = 0.2
threshold = 0.25
max_elev_diff = 10
elev_gradient = 0
num_iterations = 5
flags = titanlib.buddy_event_check(points, precip_obs, radius, num_min,
            event_threshold, threshold, max_elev_diff, elev_gradient, num_iterations)

print(flags)

[1 1 0 0]


## Climatology range check  

In [5]:
#Range check climatology
target_time = pd.to_datetime("2022-01-22T12:00:00")


unixtime = int(target_time.timestamp()) #need this as input for climatology 
tpos = np.full(points.size(), 30)
tneg = np.full(points.size(), -20)

flags = titanlib.range_check_climatology(points, observations, unixtime, tpos, tneg)
print(flags)

[0 1 0 0]


## Isolation check

In [6]:
nb_neigh = 1
radius = 2000 #m

flags = titanlib.isolation_check(points, nb_neigh, radius)
print(flags)

[0 0 1 1]


## Metadata check

In [7]:
flags = titanlib.metadata_check(points, True, True, True, True)
print(flags)

[1 1 1 1]


## Range check

In [8]:
tmin = [-20,-20,-20,-20]
tmax = [30,30,30,30]

flags = titanlib.range_check(observations, tmin, tmax)
print(flags)



[0 0 0 0]


## Spatial consistency test 

In [12]:

pos = np.full(points.size(), 4)
neg = np.full(points.size(), 8)
eps2 = np.full(points.size(), 0.5)

dataset.sct(
    5,      # num_min
    100,    # num_max
    50000,  # inner_radius
    150000, # outer_radius
    1,      # num_iterations
    20,     # num_min_prof
    200,    # min_elev_diff
    10000,  # min_horizonal_scale
    200,    # vertical_scale
    pos,
    neg,
    eps2)

print(dataset.get_flags())

[0 0 0 1]
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [11]:
## Spatial consistency test - Returned parameters Quality control flag (0=OK, 1=bad), 
# prob_gross_error	vec		Probability of gross error for each observation
# rep	vec		Coefficient of representativity

pos = np.full(points.size(), 4)
neg = np.full(points.size(), 8)
eps2 = np.full(points.size(), 0.5)

flags, prob, rep = titanlib.sct(points,
    observations,
    5,      # num_min
    100,    # num_max
    50000,  # inner_radius
    150000, # outer_radius
    5,      # num_iterations
    20,     # num_min_prof
    200,    # min_elev_diff
    10000,  # min_horizonal_scale
    200,    # vertical_scale
    pos,
    neg,
    eps2)

print(flags)
print(prob)
print(rep)

[0 0 0 0]
[0. 0. 0. 0.]
[0. 0. 0. 0.]


## Spatial consistency test resistant

In [40]:
## Spatial consistency test Resistant

pos = np.full(points.size(), 4)
neg = np.full(points.size(), 8)
eps2 = np.full(points.size(), 0.5)

obs_to_check = np.full(points.size(), 1)
background_values = np.full(points.size(), 0)
background_elab_type = titanlib.MedianOuterCircle
num_min_outer = 3
num_max_outer = 10
inner_radius = 20000
outer_radius = 50000
num_iterations = 10
num_min_prof = 1
min_elev_diff = 100
min_horizontal_scale = 250 
max_horizontal_scale = 100000
kth_closest_obs_horizontal_scale = 2
vertical_scale = 200
tpos = np.full(points.size(), 16)
tneg = np.full(points.size(), 16)
eps2 = np.full(points.size(), 0.5)
values_mina = np.full(points.size(), -20)
values_maxa = np.full(points.size(), 20)
values_minv = np.full(points.size(), 0)
values_maxv = np.full(points.size(), 10)
debug = True
basic = True


flags, scores = titanlib.sct_resistant( points, observations, obs_to_check, background_values, 
                      background_elab_type, num_min_outer, num_max_outer, 
                      inner_radius, outer_radius, num_iterations, num_min_prof, 
                      min_elev_diff, min_horizontal_scale, max_horizontal_scale, 
                      kth_closest_obs_horizontal_scale, vertical_scale, 
                      values_mina, values_maxa, values_minv, values_maxv, 
                      eps2, tpos, tneg, debug, basic)


mean t: -7.94667
mean t: -3.03333
mean t: 28.88
mean t: -7.94667
mean t: -7.954
mean t: -3.03333
mean t: 28.88
mean t: -7.94667
mean t: -7.954
mean t: -3.03333
mean t: 28.88
mean t: -7.94667
mean t: -7.954
mean t: -3.03333
mean t: 28.88
=============================================== 4
Number of observations to test is 4
Min and max acceptable values are -20 20
g lats lons elevs obs:
0 57.2 10.3 50 20
1 57.21 10.31 51 30
2 50.5 13.2 912 18
3 11.2 29.5 31 25
 +++++ Iteration 0 ++++++++++++++++
===> curr 0 ===============
p_dist  2
p_outer inner test 2 2 2
@@isolated (outer) 0
===> curr 1 ===============
p_dist  2
p_outer inner test 2 2 2
@@isolated (outer) 1
===> curr 2 ===============
p_dist  1
p_outer inner test 1 1 1
@@isolated (outer) 2
===> curr 3 ===============
p_dist  1
p_outer inner test 1 1 1
@@isolated (outer) 3
SCT loop - Removing 0 observations. Number of OI 0 time=2.38419e-05 secs
Stopping early after 1 iterations
Remaining 0 observations set to "good" 
 +++++ check on obs

## First guess test

In [41]:

lats = np.array([64.4, 64.5, 64.6])
lons = np.array([10.1, 10.1, 10.1])
elev = [200.5, 200, 220]
temp_obs = np.array([8.5, 9, 10])
points = titanlib.Points(lats, lons, elev)
N = len(lats)
obs_to_check = np.ones(N)
background_values = np.zeros(N)
background_uncertainties = np.ones(N)
background_elab_type = titanlib.MedianOuterCircle
num_min_outer = 3
num_max_outer = 10
inner_radius = 20000
outer_radius = 50000
num_iterations = 10
num_min_prof = 0
min_elev_diff = 100
min_horizontal_scale = 250
max_horizontal_scale = 100000
kth_closest_obs_horizontal_scale = 2
tpos = np.ones(N) * 5
tneg = np.ones(N) * 5
values_mina = temp_obs - 20
values_maxa = temp_obs + 20
values_minv = temp_obs - 1
values_maxv = temp_obs + 1
debug = True
basic = True
res = titanlib.fgt(points, temp_obs, obs_to_check,
    background_values, 
    background_uncertainties, 
    background_elab_type, 
    num_min_outer, 
    num_max_outer, 
    inner_radius, 
    outer_radius, 
    num_iterations, 
    num_min_prof, 
    min_elev_diff, 
    values_mina, 
    values_maxa, 
    values_minv, 
    values_maxv, 
    tpos, 
    tneg, 
    debug, 
    basic   
)

print(res)

[array([0, 0, 0], dtype=int32), array([-999., -999., -999.], dtype=float32)]
=============================================== 3
Number of observations to test is 3
Min and max acceptable values are -11.5 30
g lats lons obs:
0 64.4 10.1 8.5
1 64.5 10.1 9
2 64.6 10.1 10
 +++++ Iteration 0 ++++++++++++++++
===> curr 0 ===============
p_dist  3
p_outer inner test 3 2 2
curr lats lons obs:
0 64.4 10.1 8.5
indices_global_outer - i g lats lons obs:
0 0 64.4 10.1 8.5 -999
1 1 64.5 10.1 9 -999
2 2 64.6 10.1 10 -999
outer - i lats lons obs:
0 64.4 10.1 8.5 -999
1 64.5 10.1 9 -999
2 64.6 10.1 10 -999
indices_outer_inner - l i lats lons obs:
0 0 64.4 10.1 8.5 -999
1 1 64.5 10.1 9 -999
indices_outer_test - l m lats lons obs:
0 0 64.4 10.1 8.5 -999
1 1 64.5 10.1 9 -999
indices_inner_test - l m lats lons obs:
0 0 64.4 10.1 8.5 -999
1 1 64.5 10.1 9 -999
... background ok ...
 small_innov - index 0
 small_innov - index 1
===> curr 1 ===============
..observation not suitable as a centroid 1
===> curr 2 

## Spatial consistency test dual

<div class="alert alert-danger"> 
I haven't gotten this next bit working

In [42]:
N = len(lats)
obs_to_check = np.ones(N)
event_thresholds = 0.1
test_thresholds = 0.8
#condition = titanlib.Gt

condition = titanlib.MedianOuterCircle

num_min_outer = 3
num_max_outer = 10
inner_radius = 20000
outer_radius = 50000
num_iterations = 10
min_horizontal_scale = 250 
max_horizontal_scale = 100000
kth_closest_obs_horizontal_scale = 2
vertical_scale = 200
debug = True
titanlib.sct_dual(points, temp_obs, obs_to_check, event_thresholds, condition,
                 num_min_outer, num_max_outer, inner_radius, outer_radius,
                 num_iterations, min_horizontal_scale, max_horizontal_scale,
                 kth_closest_obs_horizontal_scale, vertical_scale,
                 test_thresholds, debug)

TypeError: Could not convert input to 1D array of type 'float'